# HAR CNN training 

In [1]:
# Imports
import numpy as np
import os
from utils.semg_utilities import *
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

## Prepare data

In [2]:
path = os.path.abspath(os.getcwd())
X_train, labels_train, list_ch_train = read_data(data_path= path + os.sep + "sEMG" + os.sep + "Database 2" + os.sep, split_type="train") # train
X_test, labels_test, list_ch_test = read_data(data_path= path + os.sep + "sEMG" + os.sep + "Database 2" + os.sep, split_type="test") # test

assert list_ch_train == list_ch_test, "Mistmatch in channels!"

In [3]:
# Normalize?
X_train, X_test = standardize(X_train, X_test)

Train/Validation Split

In [4]:
X_tr, X_vld, lab_tr, lab_vld = train_test_split(X_train, labels_train, 
                                                stratify = labels_train, random_state = 123)

One-hot encoding:

In [5]:
y_tr = one_hot(lab_tr)
y_vld = one_hot(lab_vld)
y_test = one_hot(labels_test)

In [6]:
# Imports
import tensorflow as tf

### Hyperparameters

In [7]:
batch_size = 25       # Batch size
seq_len = 2496          # Number of steps
learning_rate = 0.0001
epochs = 1000

n_classes = 6
n_channels = 2

### Construct the graph
Placeholders

In [8]:
graph = tf.Graph()

# Construct placeholders
with graph.as_default():
    inputs_ = tf.placeholder(tf.float32, [None, seq_len, n_channels], name = 'inputs')
    labels_ = tf.placeholder(tf.float32, [None, n_classes], name = 'labels')
    keep_prob_ = tf.placeholder(tf.float32, name = 'keep')
    learning_rate_ = tf.placeholder(tf.float32, name = 'learning_rate')

Build Convolutional Layers

Note: Should we use a different activation? Like tf.nn.tanh?

In [9]:
with graph.as_default():
    # (batch, 2496, 2) --> (batch, 1248, 4)
    conv1 = tf.layers.conv1d(inputs=inputs_, filters=4, kernel_size=2, strides=1, 
                             padding='same', activation = tf.nn.relu)
    max_pool_1 = tf.layers.max_pooling1d(inputs=conv1, pool_size=2, strides=2, padding='same')
    
    # (batch, 1248, 4) --> (batch, 624, 8)
    conv2 = tf.layers.conv1d(inputs=max_pool_1, filters=8, kernel_size=2, strides=1, 
                             padding='same', activation = tf.nn.relu)
    max_pool_2 = tf.layers.max_pooling1d(inputs=conv2, pool_size=2, strides=2, padding='same')
    
    # (batch, 624, 8) --> (batch, 312, 16)
    conv3 = tf.layers.conv1d(inputs=max_pool_2, filters=16, kernel_size=2, strides=1, 
                             padding='same', activation = tf.nn.relu)
    max_pool_3 = tf.layers.max_pooling1d(inputs=conv3, pool_size=2, strides=2, padding='same')
    
    # (batch, 312, 16) --> (batch, 156, 32)
    conv4 = tf.layers.conv1d(inputs=max_pool_3, filters=32, kernel_size=2, strides=1, 
                             padding='same', activation = tf.nn.relu)
    max_pool_4 = tf.layers.max_pooling1d(inputs=conv4, pool_size=2, strides=2, padding='same')

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead


Now, flatten and pass to the classifier

In [10]:
with graph.as_default():
    # Flatten and add dropout
    flat = tf.reshape(max_pool_4, (-1, 32 * 156))
    flat = tf.nn.dropout(flat, keep_prob=keep_prob_)
    
    # Predictions
    logits = tf.layers.dense(flat, n_classes)
    
    # Cost function and optimizer
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels_))
    optimizer = tf.train.AdamOptimizer(learning_rate_).minimize(cost)
    
    # Accuracy
    correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(labels_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



### Train the network

In [11]:
if (os.path.exists('checkpoints-cnn') == False):
    !mkdir checkpoints-cnn

In [ ]:
validation_acc = []
validation_loss = []

train_acc = []
train_loss = []

with graph.as_default():
    saver = tf.train.Saver()

with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    iteration = 1
   
    # Loop over epochs
    for e in range(epochs):
        
        # Loop over batches
        for x,y in get_batches(X_tr, y_tr, batch_size):
            
            # Feed dictionary
            feed = {inputs_ : x, labels_ : y, keep_prob_ : 0.5, learning_rate_ : learning_rate}
            
            # Loss
            loss, _ , acc = sess.run([cost, optimizer, accuracy], feed_dict = feed)
            train_acc.append(acc)
            train_loss.append(loss)
            
            # Print at each 5 iters
            if (iteration % 5 == 0):
                print("Epoch: {}/{}".format(e, epochs),
                      "Iteration: {:d}".format(iteration),
                      "Train loss: {:6f}".format(loss),
                      "Train acc: {:.6f}".format(acc))
            
            # Compute validation loss at every 10 iterations
            if (iteration%10 == 0):                
                val_acc_ = []
                val_loss_ = []
                
                for x_v, y_v in get_batches(X_vld, y_vld, batch_size):
                    # Feed
                    feed = {inputs_ : x_v, labels_ : y_v, keep_prob_ : 1.0}  
                    
                    # Loss
                    loss_v, acc_v = sess.run([cost, accuracy], feed_dict = feed)                    
                    val_acc_.append(acc_v)
                    val_loss_.append(loss_v)
                
                # Print info
                print("Epoch: {}/{}".format(e, epochs),
                      "Iteration: {:d}".format(iteration),
                      "Validation loss: {:6f}".format(np.mean(val_loss_)),
                      "Validation acc: {:.6f}".format(np.mean(val_acc_)))
                
                # Store
                validation_acc.append(np.mean(val_acc_))
                validation_loss.append(np.mean(val_loss_))
            
            # Iterate 
            iteration += 1
    
    saver.save(sess,"checkpoints-cnn/har.ckpt")

Epoch: 0/1000 Iteration: 5 Train loss: 1.811453 Train acc: 0.160000
Epoch: 0/1000 Iteration: 10 Train loss: 1.889068 Train acc: 0.080000
Epoch: 0/1000 Iteration: 10 Validation loss: 1.765758 Validation acc: 0.166667
Epoch: 0/1000 Iteration: 15 Train loss: 1.798224 Train acc: 0.200000
Epoch: 0/1000 Iteration: 20 Train loss: 1.857522 Train acc: 0.080000
Epoch: 0/1000 Iteration: 20 Validation loss: 1.743162 Validation acc: 0.170000
Epoch: 0/1000 Iteration: 25 Train loss: 1.835288 Train acc: 0.120000
Epoch: 0/1000 Iteration: 30 Train loss: 1.667589 Train acc: 0.200000
Epoch: 0/1000 Iteration: 30 Validation loss: 1.721933 Validation acc: 0.176667
Epoch: 0/1000 Iteration: 35 Train loss: 1.738193 Train acc: 0.200000
Epoch: 1/1000 Iteration: 40 Train loss: 1.635310 Train acc: 0.280000
Epoch: 1/1000 Iteration: 40 Validation loss: 1.705533 Validation acc: 0.186667
Epoch: 1/1000 Iteration: 45 Train loss: 1.832298 Train acc: 0.160000
Epoch: 1/1000 Iteration: 50 Train loss: 1.775885 Train acc: 0.08

Epoch: 10/1000 Iteration: 390 Train loss: 1.033943 Train acc: 0.640000
Epoch: 10/1000 Iteration: 390 Validation loss: 1.050421 Validation acc: 0.653333
Epoch: 10/1000 Iteration: 395 Train loss: 0.944905 Train acc: 0.640000
Epoch: 11/1000 Iteration: 400 Train loss: 0.898702 Train acc: 0.720000
Epoch: 11/1000 Iteration: 400 Validation loss: 1.024589 Validation acc: 0.676667
Epoch: 11/1000 Iteration: 405 Train loss: 1.105674 Train acc: 0.560000
Epoch: 11/1000 Iteration: 410 Train loss: 1.028853 Train acc: 0.640000
Epoch: 11/1000 Iteration: 410 Validation loss: 1.003615 Validation acc: 0.666667
Epoch: 11/1000 Iteration: 415 Train loss: 0.884368 Train acc: 0.800000
Epoch: 11/1000 Iteration: 420 Train loss: 1.052641 Train acc: 0.600000
Epoch: 11/1000 Iteration: 420 Validation loss: 0.985784 Validation acc: 0.706667
Epoch: 11/1000 Iteration: 425 Train loss: 0.816806 Train acc: 0.720000
Epoch: 11/1000 Iteration: 430 Train loss: 1.063629 Train acc: 0.640000
Epoch: 11/1000 Iteration: 430 Validat

Epoch: 21/1000 Iteration: 765 Train loss: 0.809559 Train acc: 0.800000
Epoch: 21/1000 Iteration: 770 Train loss: 0.571925 Train acc: 0.880000
Epoch: 21/1000 Iteration: 770 Validation loss: 0.671356 Validation acc: 0.786667
Epoch: 21/1000 Iteration: 775 Train loss: 0.570414 Train acc: 0.680000
Epoch: 21/1000 Iteration: 780 Train loss: 0.703041 Train acc: 0.800000
Epoch: 21/1000 Iteration: 780 Validation loss: 0.667537 Validation acc: 0.790000
Epoch: 21/1000 Iteration: 785 Train loss: 0.436593 Train acc: 0.880000
Epoch: 21/1000 Iteration: 790 Train loss: 0.642532 Train acc: 0.800000
Epoch: 21/1000 Iteration: 790 Validation loss: 0.667375 Validation acc: 0.786667
Epoch: 22/1000 Iteration: 795 Train loss: 0.554160 Train acc: 0.760000
Epoch: 22/1000 Iteration: 800 Train loss: 0.615676 Train acc: 0.880000
Epoch: 22/1000 Iteration: 800 Validation loss: 0.657825 Validation acc: 0.793333
Epoch: 22/1000 Iteration: 805 Train loss: 0.513533 Train acc: 0.800000
Epoch: 22/1000 Iteration: 810 Train l

Epoch: 31/1000 Iteration: 1135 Train loss: 0.367390 Train acc: 0.880000
Epoch: 31/1000 Iteration: 1140 Train loss: 0.601155 Train acc: 0.800000
Epoch: 31/1000 Iteration: 1140 Validation loss: 0.560928 Validation acc: 0.793333
Epoch: 31/1000 Iteration: 1145 Train loss: 0.312723 Train acc: 0.960000
Epoch: 31/1000 Iteration: 1150 Train loss: 0.403097 Train acc: 0.840000
Epoch: 31/1000 Iteration: 1150 Validation loss: 0.563433 Validation acc: 0.806667
Epoch: 32/1000 Iteration: 1155 Train loss: 0.408912 Train acc: 0.840000
Epoch: 32/1000 Iteration: 1160 Train loss: 0.512067 Train acc: 0.840000
Epoch: 32/1000 Iteration: 1160 Validation loss: 0.559797 Validation acc: 0.803333
Epoch: 32/1000 Iteration: 1165 Train loss: 0.369362 Train acc: 0.880000
Epoch: 32/1000 Iteration: 1170 Train loss: 0.353373 Train acc: 0.880000
Epoch: 32/1000 Iteration: 1170 Validation loss: 0.554622 Validation acc: 0.790000
Epoch: 32/1000 Iteration: 1175 Train loss: 0.541302 Train acc: 0.800000
Epoch: 32/1000 Iteration

Epoch: 41/1000 Iteration: 1505 Train loss: 0.260989 Train acc: 0.920000
Epoch: 41/1000 Iteration: 1510 Train loss: 0.349315 Train acc: 0.920000
Epoch: 41/1000 Iteration: 1510 Validation loss: 0.506521 Validation acc: 0.813333
Epoch: 42/1000 Iteration: 1515 Train loss: 0.313531 Train acc: 0.920000
Epoch: 42/1000 Iteration: 1520 Train loss: 0.292216 Train acc: 0.920000
Epoch: 42/1000 Iteration: 1520 Validation loss: 0.503360 Validation acc: 0.810000
Epoch: 42/1000 Iteration: 1525 Train loss: 0.382875 Train acc: 0.840000
Epoch: 42/1000 Iteration: 1530 Train loss: 0.293943 Train acc: 0.920000
Epoch: 42/1000 Iteration: 1530 Validation loss: 0.499934 Validation acc: 0.803333
Epoch: 42/1000 Iteration: 1535 Train loss: 0.442167 Train acc: 0.840000
Epoch: 42/1000 Iteration: 1540 Train loss: 0.226701 Train acc: 0.920000
Epoch: 42/1000 Iteration: 1540 Validation loss: 0.502242 Validation acc: 0.806667
Epoch: 42/1000 Iteration: 1545 Train loss: 0.380566 Train acc: 0.880000
Epoch: 43/1000 Iteration

...

Epoch: 950/1000 Iteration: 34220 Validation loss: 0.875069 Validation acc: 0.833333
Epoch: 950/1000 Iteration: 34225 Train loss: 0.001812 Train acc: 1.000000
Epoch: 950/1000 Iteration: 34230 Train loss: 0.000977 Train acc: 1.000000
Epoch: 950/1000 Iteration: 34230 Validation loss: 0.863626 Validation acc: 0.830000
Epoch: 950/1000 Iteration: 34235 Train loss: 0.001102 Train acc: 1.000000
Epoch: 951/1000 Iteration: 34240 Train loss: 0.000031 Train acc: 1.000000
Epoch: 951/1000 Iteration: 34240 Validation loss: 0.861204 Validation acc: 0.843333
Epoch: 951/1000 Iteration: 34245 Train loss: 0.002822 Train acc: 1.000000
Epoch: 951/1000 Iteration: 34250 Train loss: 0.013823 Train acc: 1.000000
Epoch: 951/1000 Iteration: 34250 Validation loss: 0.908612 Validation acc: 0.840000
Epoch: 951/1000 Iteration: 34255 Train loss: 0.000043 Train acc: 1.000000
Epoch: 951/1000 Iteration: 34260 Train loss: 0.000602 Train acc: 1.000000
Epoch: 951/1000 Iteration: 34260 Validation loss: 0.893202 Validation ac

Epoch: 960/1000 Iteration: 34580 Train loss: 0.001179 Train acc: 1.000000
Epoch: 960/1000 Iteration: 34580 Validation loss: 0.841207 Validation acc: 0.846667
Epoch: 960/1000 Iteration: 34585 Train loss: 0.002736 Train acc: 1.000000
Epoch: 960/1000 Iteration: 34590 Train loss: 0.000131 Train acc: 1.000000
Epoch: 960/1000 Iteration: 34590 Validation loss: 0.829678 Validation acc: 0.846667
Epoch: 960/1000 Iteration: 34595 Train loss: 0.000468 Train acc: 1.000000
Epoch: 961/1000 Iteration: 34600 Train loss: 0.000647 Train acc: 1.000000
Epoch: 961/1000 Iteration: 34600 Validation loss: 0.840620 Validation acc: 0.846667
Epoch: 961/1000 Iteration: 34605 Train loss: 0.000200 Train acc: 1.000000
Epoch: 961/1000 Iteration: 34610 Train loss: 0.000847 Train acc: 1.000000
Epoch: 961/1000 Iteration: 34610 Validation loss: 0.845247 Validation acc: 0.846667
Epoch: 961/1000 Iteration: 34615 Train loss: 0.000113 Train acc: 1.000000
Epoch: 961/1000 Iteration: 34620 Train loss: 0.001243 Train acc: 1.00000

Epoch: 970/1000 Iteration: 34940 Train loss: 0.000408 Train acc: 1.000000
Epoch: 970/1000 Iteration: 34940 Validation loss: 0.789105 Validation acc: 0.850000
Epoch: 970/1000 Iteration: 34945 Train loss: 0.000604 Train acc: 1.000000
Epoch: 970/1000 Iteration: 34950 Train loss: 0.000925 Train acc: 1.000000
Epoch: 970/1000 Iteration: 34950 Validation loss: 0.780001 Validation acc: 0.850000
Epoch: 970/1000 Iteration: 34955 Train loss: 0.000411 Train acc: 1.000000
Epoch: 971/1000 Iteration: 34960 Train loss: 0.000969 Train acc: 1.000000
Epoch: 971/1000 Iteration: 34960 Validation loss: 0.787193 Validation acc: 0.850000
Epoch: 971/1000 Iteration: 34965 Train loss: 0.000303 Train acc: 1.000000
Epoch: 971/1000 Iteration: 34970 Train loss: 0.000941 Train acc: 1.000000
Epoch: 971/1000 Iteration: 34970 Validation loss: 0.859804 Validation acc: 0.836667
Epoch: 971/1000 Iteration: 34975 Train loss: 0.000214 Train acc: 1.000000
Epoch: 971/1000 Iteration: 34980 Train loss: 0.000360 Train acc: 1.00000

Epoch: 980/1000 Iteration: 35290 Validation loss: 0.780483 Validation acc: 0.850000
Epoch: 980/1000 Iteration: 35295 Train loss: 0.000639 Train acc: 1.000000
Epoch: 980/1000 Iteration: 35300 Train loss: 0.000153 Train acc: 1.000000
Epoch: 980/1000 Iteration: 35300 Validation loss: 0.780203 Validation acc: 0.846667
Epoch: 980/1000 Iteration: 35305 Train loss: 0.001964 Train acc: 1.000000
Epoch: 980/1000 Iteration: 35310 Train loss: 0.000269 Train acc: 1.000000
Epoch: 980/1000 Iteration: 35310 Validation loss: 0.779367 Validation acc: 0.846667
Epoch: 980/1000 Iteration: 35315 Train loss: 0.000355 Train acc: 1.000000
Epoch: 981/1000 Iteration: 35320 Train loss: 0.000171 Train acc: 1.000000
Epoch: 981/1000 Iteration: 35320 Validation loss: 0.780848 Validation acc: 0.850000
Epoch: 981/1000 Iteration: 35325 Train loss: 0.001374 Train acc: 1.000000
Epoch: 981/1000 Iteration: 35330 Train loss: 0.005556 Train acc: 1.000000
Epoch: 981/1000 Iteration: 35330 Validation loss: 0.787602 Validation ac

Epoch: 990/1000 Iteration: 35645 Train loss: 0.000040 Train acc: 1.000000
Epoch: 990/1000 Iteration: 35650 Train loss: 0.000130 Train acc: 1.000000
Epoch: 990/1000 Iteration: 35650 Validation loss: 0.786843 Validation acc: 0.850000
Epoch: 990/1000 Iteration: 35655 Train loss: 0.000464 Train acc: 1.000000
Epoch: 990/1000 Iteration: 35660 Train loss: 0.000673 Train acc: 1.000000
Epoch: 990/1000 Iteration: 35660 Validation loss: 0.790223 Validation acc: 0.850000
Epoch: 990/1000 Iteration: 35665 Train loss: 0.002113 Train acc: 1.000000
Epoch: 990/1000 Iteration: 35670 Train loss: 0.000158 Train acc: 1.000000
Epoch: 990/1000 Iteration: 35670 Validation loss: 0.794967 Validation acc: 0.853333
Epoch: 990/1000 Iteration: 35675 Train loss: 0.000291 Train acc: 1.000000
Epoch: 991/1000 Iteration: 35680 Train loss: 0.008976 Train acc: 1.000000
Epoch: 991/1000 Iteration: 35680 Validation loss: 0.795841 Validation acc: 0.853333
Epoch: 991/1000 Iteration: 35685 Train loss: 0.000546 Train acc: 1.00000

In [ ]:
# Plot training and test loss
t = np.arange(iteration-1)

plt.figure(figsize = (6,6))
plt.plot(t, np.array(train_loss), 'r-', t[t % 10 == 0], np.array(validation_loss), 'b*')
plt.xlabel("iteration")
plt.ylabel("Loss")
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
# Plot Accuracies
plt.figure(figsize = (6,6))

plt.plot(t, np.array(train_acc), 'r-', t[t % 10 == 0], validation_acc, 'b*')
plt.xlabel("iteration")
plt.ylabel("Accuray")
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

## Evaluate on test set

In [ ]:
test_acc = []

with tf.Session(graph=graph) as sess:
    # Restore
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints-cnn'))
    
    for x_t, y_t in get_batches(X_test, y_test, batch_size):
        feed = {inputs_: x_t,
                labels_: y_t,
                keep_prob_: 1}
        
        batch_acc = sess.run(accuracy, feed_dict=feed)
        test_acc.append(batch_acc)
    print("Test accuracy: {:.6f}".format(np.mean(test_acc)))